In [1]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi')

In [2]:
%%px
# Import the libraries

import os
import sys
import math
import pyJHTDB
import numpy as np
import pyfftw as ft 
from mpi4py import MPI
import matplotlib
import matplotlib.pyplot as plt
from pyJHTDB import libJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse

from mpiFFT4py.slab import R2C

In [3]:
%%px

Nx = isotropic1024coarse['nx']; Ny = isotropic1024coarse['ny']; Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']; Ly = isotropic1024coarse['ly']; Lz = isotropic1024coarse['lz']

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nproc = comm.Get_size()
if(rank==0):
    print("n_proc = "+str(nproc))
    print("rank = "+str(rank))

# Computational Domain

nx=Nx//nproc; ny=Ny; nz=Nz
nz_half=nz//2
nek=int(math.sqrt(2.0)/3*Nx)
time = 0.0

chkSz = 32
slabs = nx//chkSz

[stdout:7] 
n_proc = 8
rank = 0


In [4]:
%%px 

N = np.array([Nx,Ny,Nz],dtype=int)
L = np.array([Lx,Ly,Lz],dtype=float)

FFT = R2C(N, L, MPI.COMM_WORLD, "double", communication='Alltoallw')

In [5]:
%%px

cacheEnstrophyData = False
loadEnstrophyFromCache = True

folder = "/home/idies/workspace/scratch"
filename = "ref-enstrophy-"+str(rank)+".npz"
file = folder + "/" + filename

if(loadEnstrophyFromCache):
    comm.Barrier(); t1=MPI.Wtime()
    content = np.load(file)
    
    w2 = ft.zeros_aligned(FFT.real_shape(), dtype=FFT.float)
    
    if(int(content['nproc'])!=nproc):
        print("Unmatched number of processes. Must first pre-process to adequate number of process")
    w2[:,:,:] = content['w2'].astype(FFT.float)
    
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))

if(cacheEnstrophyData):
    
    comm.Barrier(); t1=MPI.Wtime()
    np.savez(file,w2=w2,nproc=nproc)
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Caching the data: {0:.2f} seconds\n'.format(t2-t1))

[stdout:7] 
Finished loading
Load from disk: 194.77 seconds


In [6]:
%%px

avgOmega = np.average(w2)
avgOmegaGl=np.zeros(1,dtype=FFT.float)

comm.Reduce([avgOmega,MPI.DOUBLE],[avgOmegaGl,MPI.DOUBLE],op=MPI.SUM)
avgOmega = avgOmegaGl[0]/nproc
avgOmega = comm.bcast(avgOmega, root=0)

##########################

minw2 = w2.min()
maxw2 = w2.max()

minwGl=np.zeros(nproc,dtype=FFT.float)
maxwGl=np.zeros(nproc,dtype=FFT.float)

comm.Allgather([minw2,MPI.DOUBLE],[minwGl,MPI.DOUBLE])
comm.Allgather([maxw2,MPI.DOUBLE],[maxwGl,MPI.DOUBLE])

minw2 = minwGl.min()
maxw2 = maxwGl.max()

comm.Barrier()

if rank==0:
    print("<w^2> : "+str(avgOmega))
    print("min w2/<w^2> : "+str(minw2/avgOmega))
    print("min w2/<w^2> : "+str(maxw2/avgOmega))

[stdout:7] 
<w^2> : 498.435342491
min w2/<w^2> : 1.43020513214e-07
min w2/<w^2> : 675.170320624


In [7]:
%%px

dx = isotropic1024coarse['dx']
ner = int(1024*np.sqrt(3))
rbins = np.linspace(-0.5*dx,2*np.pi*np.sqrt(3)+0.5*dx,ner+1)

comm.Barrier(); t1=MPI.Wtime()

X = np.zeros(FFT.real_shape(), dtype=FFT.float)
Y = np.zeros(FFT.real_shape(), dtype=FFT.float)
Z = np.zeros(FFT.real_shape(), dtype=FFT.float)

r2 = np.zeros(FFT.real_shape(), dtype=FFT.float)

chi = np.zeros(FFT.real_shape(), dtype=FFT.float)
cchi = np.zeros(FFT.complex_shape(), dtype=FFT.complex)
corr = np.zeros(FFT.real_shape(), dtype=FFT.float)
iCorr = np.zeros(FFT.real_shape(), dtype=FFT.float)

corrSum = np.zeros(FFT.real_shape(), dtype=FFT.float)
corrF = np.zeros(FFT.real_shape(), dtype=FFT.float)

r2Sum = np.zeros(FFT.real_shape(), dtype=FFT.float)
r2F = np.zeros(FFT.real_shape(), dtype=FFT.float)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Alocating vectors: {0:.2f} seconds\n'.format(t2-t1))

[stdout:7] Alocating vectors: 0.03 seconds


In [8]:
%%px

comm.Barrier(); t1=MPI.Wtime()
for i in range(nx):
    if (i+rank*nx < Nx//2):
        X[i,:,:] = (i+rank*nx)*isotropic1024coarse['dx']
    else:
        X[i,:,:] = Lx-(i+rank*nx)*isotropic1024coarse['dx']

for j in range(ny):
    if (j < Ny//2):
        Y[:,j,:] = j*isotropic1024coarse['dy']
    else:
        Y[:,j,:] = Ly-j*isotropic1024coarse['dy']
    
for k in range(nz):
    if (k < Nz//2):
        Z[:,:,k] = k*isotropic1024coarse['dz']
    else:
        Z[:,:,k] = Lz-k*isotropic1024coarse['dz']
    
r2[:,:,:] = X[:,:,:]**2+Y[:,:,:]**2+Z[:,:,:]**2

r2rt = np.sqrt(r2)
comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Preparing the real domain for radial integration: {0:.2f} seconds\n'.format(t2-t1))

[stdout:7] Preparing the real domain for radial integration: 423.83 seconds


In [ ]:
%%px 
%matplotlib inline

t = [1]#,2,3,4,5,6]
c = ['r']#,'g','b','c','y','m']

hist = []
r2st = []

for i in range(1):
    if(rank==0):
        print("Treshold = "+str(t[i]))
        
    #######################################
    comm.Barrier(); t1=MPI.Wtime()
    
    chi[:,:,:] = 0
    chi[w2 > t[i]*avgOmega] = 1
    
    #######################################
    
    comm.Barrier(); t2=MPI.Wtime()
    cchi = FFT.fftn(chi,cchi)
    
    tmp = cchi*(cchi.conj())
    corr = FFT.ifftn(tmp,corr)
    
    #######################################
    
    comm.Barrier(); t3=MPI.Wtime()
        
    #######################################
    
    comm.Barrier(); t4=MPI.Wtime()
    
    corrLoc,redges = np.histogram(r2rt,range=(0.5*dx,(ner+0.5)*dx),bins = rbins,weights=corr)
    
    r2Loc,r2edges = np.histogram(r2rt,range=(0.5*dx,(ner+0.5)*dx),bins = rbins)
       
    #######################################
    
    comm.Barrier(); t5=MPI.Wtime()
    
    corrLoc=np.float32(corrLoc)
    comm.Reduce([corrLoc,MPI.REAL],[corrSum,MPI.REAL],op=MPI.SUM)
    np.copyto(corrF,corrSum)
    
    corrApend=np.zeros(ner,dtype='float32') 
    corrApend[:] = corrF[:]
    
    #########################
    
    r2Loc=np.float32(r2Loc)
    comm.Reduce([r2Loc,MPI.REAL],[r2Sum,MPI.REAL],op=MPI.SUM)
    np.copyto(r2F,r2Sum)
    
    r2Apend=np.zeros(ner,dtype='float32') 
    r2Apend[:] = r2F[:]
    
    if rank==0:
        hist.append(corrApend[:])
        r2st.append(r2Apend[:])
    
    #######################################
    
    comm.Barrier(); t6=MPI.Wtime()
    
    if rank==0:
        sys.stdout.write('threshold used: {0:.2f} \n'.format(t[i]))
        sys.stdout.write('Creating the excursion set: {0:.2f} seconds\n'.format(t2-t1))
        sys.stdout.write('Computing the 2-point correlation function: {0:.2f} seconds\n'.format(t3-t2))
        sys.stdout.write('Cleaning all but the 1st octant: {0:.2f} seconds\n'.format(t4-t3))
        sys.stdout.write('Integrating radially on each slab: {0:.2f} seconds\n'.format(t5-t4))
        sys.stdout.write('Adding the results form different slabs: {0:.2f} seconds\n'.format(t6-t5))
        sys.stdout.write('Full radial 2-point correlation calculation: {0:.2f} seconds\n'.format(t6-t1))
        sys.stdout.write('\n')

In [ ]:
%%px

if rank==0:
    print(corr)

In [ ]:
%%px

if(rank==0):
    matplotlib.rc('xtick', labelsize=20) 
    matplotlib.rc('ytick', labelsize=20) 
    
    idx = r2st[0]>0
    
    rp=np.linspace(1*dx,ner*dx,ner)
    
    lrp = rp[idx]
    
    tempRp = lrp[(lrp>0.1)&(lrp<1.0)]
    plt.figure(figsize=(12,12))
    plt.xlabel(r'$r$', fontsize=20)
    plt.ylabel(r'$\frac{C(r)}{C(0)} = \frac{1}{C(0)} \int_{S^2} \langle \chi({\bf x}) \chi({\bf x}+{\bf r}) \rangle d\Omega$', fontsize=20)
    plt.xlim([7*10**(-3),np.pi])
    plt.ylim([10**(-2),2*10**0])
    plt.grid()
    
    for i in range(1):#6):        
        corrF1 = hist[i][idx]/r2st[i][idx]
        
        tempCorrF = corrF1[(lrp>0.1)&(lrp<1.0)]
        
        ctmp = (corrF1[300])/(corrF1[0])
        
        fit = np.polyfit(np.log(tempRp),np.log(tempCorrF/corrF1[0]),1)
        print("t = "+str(t[i])+"*sigma_2: Linear fit [alpha A] = "+str(fit))
        
        plt.loglog(lrp,(corrF1)/(corrF1[0]),c[i],label=r'$\omega^2 > '+str(t[i])
                                                   +r'\langle \omega^2\rangle$ : $\Gamma ='
                                                   +str(fit[0])+r'$')
        
        corrF1 = hist[i][idx]
        
        plt.loglog(lrp,np.exp(fit[1])*(lrp**(fit[0])),'k--')
        
    plt.legend(loc='lower left',prop={'size':16})
    plt.savefig('mpifft4py-twopoint-enstrophy-0.pdf')